In [4]:
import pygal
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import SVG, display
from archival_cost import Cloud, Sync

## Cloud options

In [5]:
azure_archive_LRS_east = Cloud('Azure Archive LRS', 'Canada East', 'CAD')
azure_archive_LRS_east.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0031,  0.0031,   0.0031], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.68/10000,
    retrieval_costs = 0.0308,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [7]:
azure_archive_GRS_east = Cloud('Azure Archive GRS', 'Canada East', 'CAD')
azure_archive_GRS_east.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0060,  0.0060,   0.0060], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.68/10000,
    retrieval_costs = 0.0308,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [8]:
azure_archive_LRS_central = Cloud('Azure Archive LRS', 'Canada Central', 'CAD')
azure_archive_LRS_central.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0029,  0.0029,   0.0029], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.04/10000,
    retrieval_costs = 0.0282,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [9]:
azure_archive_GRS_central = Cloud('Azure Archive GRS', 'Canada Central', 'CAD')
azure_archive_GRS_central.set_costs(
    storage_tiers = [50*1024, 500*1024, 1e31], 
    storage_costs = [0.0060,  0.0060,   0.0060], 
    manipulation_request_cost = 0.3/10000,
    retrieval_request_cost = 7.04/10000,
    retrieval_costs = 0.0282,
    outbound_tiers = [5, 10*1024, 50*1024, 150*1024, 500*1024],
    outbound_costs = [0, 0.112,   0.107,   0.090,    0.064]
)

In [10]:
amazon_glacier_canada = Cloud('Amazon S3 Glacier BULK', 'Canada Central', 'USD')
amazon_glacier_canada.set_costs(
    storage_tiers = [1e31], 
    storage_costs = [0.0045], 
    manipulation_request_cost = 0.055/1000,
    retrieval_request_cost = 0.0275/1000,
    retrieval_costs = 0.00275,
    outbound_tiers = [1, 10*1024, 50*1024, 150*1024, 1e31],
    outbound_costs = [0, 0.09,    0.085,   0.07,    0.05]
)

In [11]:
google_coldline_canada = Cloud('Google Cloud Coldline', 'Montreal', 'CAD')
google_coldline_canada.set_costs(
    storage_tiers = [1e31], 
    storage_costs = [0.007], 
    manipulation_request_cost = 0.1/10000,
    retrieval_request_cost = 0.05/10000,
    retrieval_costs = 0.05,
    outbound_tiers = [1024, 10*1024, 1e31],
    outbound_costs = [0.12, 0.11,    0.08]
)

## File Sync Options

In [12]:
teamshare = Sync('Teamshare', 'UBC', 'CAD')
teamshare.set_costs(0.15)

## Tape Archival Options

In [14]:
import numpy

### Plotting func def

In [117]:
def find_intersections(xrange, choice, choice1):
    diff = numpy.abs(choice-choice1)
    mindiff = diff.min()
    if mindiff > 10:
        return False, False
    x = numpy.where(diff == mindiff)[0][0]
    return xrange[x], choice[x]

In [9]:
def storage_cost(choices, attributes, filename, annual=False):
    if annual:
        chart = pygal.XY(xrange=(0,501), title='Archival Annual Comparison', x_title='Storage(TB)', y_title='Cost per Year (CAD)') 
    else:
        chart = pygal.XY(xrange=(0,501), title='Archival Monthly Comparison', x_title='Storage(TB)', y_title='Cost per Month (CAD)')
    x = np.arange(0, 500, 0.25)
    for choice in choices:
        choice.set_manipuation_requests(attributes['manipulation_requests'])
        choice.set_retrieval_requests(attributes['retrieval_requests'])
        choice.set_data_retrieval(attributes['data_retrieval'])
        choice.set_outbound_transfer(attributes['outbound_trasfer'])
        if annual:
            y = [(i, choice.set_storage(i).yearly_cost()) for i in x]
        else:
            y = [(i, choice.set_storage(i).monthly_cost()) for i in x]
        label = choice.name + " " + choice.location
        choice.set_data_range(y)
        chart.add(label, y)
    for choice in choices:
        
    chart.render_to_file(filename) 

## Making plots

In [10]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'cloud_annual.svg', True)

In [11]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'cloud_monthly.svg', False)

In [12]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada,
           teamshare]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'storage_monthly.svg', False)

In [13]:
choices = [azure_archive_LRS_east,
           azure_archive_GRS_east,
           azure_archive_LRS_central, 
           azure_archive_GRS_central,
           amazon_glacier_canada, 
           google_coldline_canada,
           teamshare]
attributes = {
    'manipulation_requests' : 10000,
    'retrieval_requests' : 10000,
    'data_retrieval' : 1*1024,
    'outbound_trasfer' : 1*1024
}
storage_cost(choices, attributes, 'storage_annual.svg', True)